In [ ]:
import csv
from ultralytics import YOLO

# Load a pretrained model
model = YOLO("yolo11n.pt") # Using yolov8n as an example, use your "yolo11n.pt" if that's correct

# Define the video path
video_path = "/fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/horseVid.mp4"

# Define the output CSV file path
csv_output_path = "/fs/nexus-scratch/hwahed/dlcDatasetMaker/detected_objects.csv" # Choose your desired path

# Get the mapping from class ID to class name
class_names_dict = model.names
print(f"Model class names: {class_names_dict}") # Good to print this to verify names

# Define the target classes you want to detect
target_classes = ["horse", "person"] # YOLO COCO models usually use 'person'

# Find the class IDs for your target classes
target_class_ids = []
for class_id, name in class_names_dict.items():
    if name in target_classes:
        target_class_ids.append(class_id)

if not target_class_ids:
    print(f"Warning: None of the target classes {target_classes} found in model.names.")
    print("Available classes:", class_names_dict)
    exit()
else:
    print(f"Targeting class IDs: {target_class_ids} for classes: {target_classes}")


# Open the CSV file for writing
with open(csv_output_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write the header row
    csv_writer.writerow(["frame_number", "class_name", "xc", "yc", "width", "height"])

    # Process the video using stream=True for efficiency
    # Set save=False if you only want the CSV.
    # If you also want the annotated video, set save=True and specify project/name.
    results = model(video_path, stream=True, save=False, verbose=False) # verbose=False to reduce console output

    frame_number = 0
    for result in results:
        frame_number += 1

        # Get bounding box information
        boxes = result.boxes # This is a Boxes object

        if boxes is not None and len(boxes) > 0:
            # Iterate through each detected box
            for i in range(len(boxes)):
                class_id = int(boxes.cls[i].item())

                # Check if the detected class is one of our target classes
                if class_id in target_class_ids:
                    detected_class_name = class_names_dict[class_id]

                    # Get bounding box coordinates in xywh format (center_x, center_y, width, height)
                    box_coords_xywh = boxes.xywh[i].cpu().numpy()
                    xc, yc, w, h = box_coords_xywh

                    # Write the data to the CSV file
                    csv_writer.writerow([
                        frame_number,
                        detected_class_name,
                        f"{xc:.2f}",  # Format to 2 decimal places
                        f"{yc:.2f}",
                        f"{w:.2f}",
                        f"{h:.2f}"
                    ])
        else:
            # Optionally, you could write a row indicating no target detections in this frame
            # For now, we'll just skip frames with no relevant detections
            pass

        if frame_number % 100 == 0: # Print progress every 100 frames
            print(f"Processed frame {frame_number}...")

print("-" * 60)
print(f"Processing complete. Data exported to: {csv_output_path}")
print("-" * 60)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/86) /fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/horseVid.mp4: 384x640 1 person, 1 horse, 7.7ms
video 1/1 (frame 2/86) /fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/horseVid.mp4: 384x640 1 person, 1 horse, 7.3ms
video 1/1 (frame 3/86) /fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/horseVid.mp4: 384x640 1 person, 1 horse, 7.2ms
video 1/1 (frame 4/86) /fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/horseVid.mp4: 384x6